# Function binding

In [116]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish

from langchain.tools import tool
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional
import requests
import datetime
import pytz


# Load environment variables from .env file
load_dotenv()

True

In [117]:
# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float, **kwargs) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_hour = datetime.datetime.now(pytz.timezone('America/Panama')).hour
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    current_temperature = temperature_list[current_hour]
    
    return f'The current temperature is {current_temperature}°C'

In [118]:
tools = [format_tool_to_openai_function(get_current_temperature)]
print(tools)

# {'name': 'get_current_temperature', 
#  'description': 'Fetch current temperature for given coordinates.', 
#  'parameters': {'properties': 
#                 {'latitude': {'description': 'Latitude of the location to fetch weather data for', 'type': 'number'}, 
#                  'longitude': {'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}}, 
# 'required': ['latitude', 'longitude'], 'type': 'object'}}

[{'name': 'get_current_temperature', 'description': 'Fetch current temperature for given coordinates.', 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for', 'type': 'number'}, 'longitude': {'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}}, 'required': ['latitude', 'longitude'], 'type': 'object'}}]


In [119]:
#prompt = ChatPromptTemplate.from_template("What is the weather now in {input} ?")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
model = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=150).bind(functions=tools)
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [120]:
for city in ["Medellin", "Sao pablo"]:
    output = chain.invoke({f"input":"what is the weather now in {city}".format(city=city)})
    print("Getting the weather for", city)
    print(output)
    print("Output type:", type(output))
    if not isinstance(output, AgentFinish):
        print(output.tool)
        print(output.tool_input)
        get_current_temperature(output.tool_input)
        print("###---"*30)
        print("\n")

Getting the weather for Medellin
tool='get_current_temperature' tool_input={'latitude': 6.2442, 'longitude': -75.5812} log="\nInvoking: `get_current_temperature` with `{'latitude': 6.2442, 'longitude': -75.5812}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":6.2442,"longitude":-75.5812}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 86, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'function_call', 'logprobs': None}, id='run-0529af5b-dc9e-4054-94a7-7426b95cb048-0')]
Output type: <class 'langchain_core.agents.AgentActionMessageLog'>
get_current_temperature
{'latitude': 6.2442, 'longitude': -75.5812}
###--

In [121]:
output = chain.invoke({"input":"hello from python/ml community"})
if not isinstance(output, AgentFinish):
    print(output.tool)
    print(output.tool_input)
else:
    print(output)

return_values={'output': 'Hello, Python/ML enthusiast! 🐍🤖 How can I assist you today? Are you looking for some spicy machine learning tips or perhaps a Pythonic joke?'} log='Hello, Python/ML enthusiast! 🐍🤖 How can I assist you today? Are you looking for some spicy machine learning tips or perhaps a Pythonic joke?'
